# (Purpose) Build a machine learning SVM classifier

In [ ]:
SCALE_DATA = False    
# SCALE_DATA = True 

# ---------------------- TCGA dataset ---------------------------              
PATH_TO_TCGA_DATA  = "data/preprocessing_combinations/tcga_unscaled_unnormalized_nobatchcorrection__mockData.tsv"                 
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_unnormalized_nobatchcorrection.tsv"  # realData0
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_unnormalized_batchcorrected.tsv"     # realData1 
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_unnormalized_batchcorrected_protocol.tsv"      
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_unnormalized_batchcorrected_disease.tsv"     
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_unnormalized_batchcorrected_consortium.tsv"   
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_unnormalized_batchcorrected_protocoldisease.tsv"   
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_unnormalized_batchcorrected_protocolconsortium.tsv"   
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_unnormalized_batchcorrected_diseaseconsortium.tsv"   
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_qn_nobatchcorrection.tsv"            # realData2
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_qn_batchcorrected.tsv"               # realData3
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_qntarget_nobatchcorrection.tsv"      # realData4
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_qntarget_batchcorrected.tsv"         # realData5
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_fsqn_nobatchcorrection.tsv"          # realData6
# PATH_TO_TCGA_DATA = "data/preprocessing_combinations/tcga_unscaled_fsqn_batchcorrected.tsv"             # realData7    
    
    
# ---------------------- GTEx dataset ---------------------------     
PATH_TO_GTEX_DATA  = "data/preprocessing_combinations/gtex_unscaled_unnormalized_nobatchcorrection__mockData.tsv"                 
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_unnormalized_nobatchcorrection.tsv"  # realData0             
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_unnormalized_batchcorrected.tsv"     # realData1            
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_unnormalized_batchcorrected_protocol.tsv"                
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_unnormalized_batchcorrected_disease.tsv"              
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_unnormalized_batchcorrected_consortium.tsv" 
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_unnormalized_batchcorrected_protocoldisease.tsv"   
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_unnormalized_batchcorrected_protocolconsortium.tsv"   
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_unnormalized_batchcorrected_diseaseconsortium.tsv"                    
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_qn_nobatchcorrection.tsv"            # realData2
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_qn_batchcorrected.tsv"               # realData3
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_qntarget_nobatchcorrection.tsv"      # realData4 
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_qntarget_batchcorrected.tsv"         # realData5             
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_fsqn_nobatchcorrection.tsv"          # realData6
# PATH_TO_GTEX_DATA = "data/preprocessing_combinations/gtex_unscaled_fsqn_batchcorrected.tsv"             # realData7                       

In [ ]:
SEED_FOR_RANDOM_STATE = 12345
MAX_JOBS = 20          # use -3; this uses all possible except for 2 cores; if use (-1) then all cores used

EXPLAINED_VARIANCE_PERCENT = 80

# SCALER_TYPE = "StandardScaler"
SCALER_TYPE = "MinMaxScaler"

PATH_TO_UMAP_PLOTS   = "results/classification_UMAP_PLOTS/" + current_date 
PATH_TO_UMAP_PLOTS_WITHOUT_A_TITLE   = "results/classification_UMAP_PLOTS/pdfs_without_a_title/" + current_date 
PATH_TO_PCA_PLOTS   = "results/classification_PCA_PLOTS/" + current_date 
PATH_TO_PCA_PLOTS_WITHOUT_A_TITLE   = "results/classification_PCA_PLOTS/pdfs_without_a_title/" + current_date 
PATH_TO_CV_RESULTS  = "results/classification_CV_RESULTS/" + current_date 
PATH_TO_PREDICTIONS = "results/classification_PREDICTIONS/" + current_date 
PATH_TO_PREDICTIONS_EXCEL_FILES = "results/classification_PREDICTIONS/excel_files/" + current_date 

## Import packages required in Python

In [ ]:
print("  begin -- importing Python packages")

import numpy as np              
import pandas as pd             
import matplotlib               
import matplotlib.cm            
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator
import joblib                   
        
import os                       
import umap.umap_ as umap       
import openpyxl                 

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder            
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef                
                                

                
    
print("  done  -- importing Python packages")

In [ ]:
print("  Checking the number of cores in the current system (will be using all)")    
n_cpu = os.cpu_count()
print("    number of CPUs in the system:", n_cpu)
print("    number of CPUS will use during grid search:", MAX_JOBS)

# print("  Enabling 'forkserver' option for multithreading to prevent freezes with too many cores")
# # (source) https://scikit-learn.org/stable/faq.html#why-do-i-sometime-get-a-crash-freeze-with-n-jobs-1-under-osx-or-linux
# multiprocessing.set_start_method('forkserver')

## Load data

In [ ]:
print("  Trying to load tcga data (** takes about 7 minutes **)")  
df = None

print("[TIMESTAMP]", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
try:
    df = pd.read_table(PATH_TO_TCGA_DATA)
    
    print ("    File(s) found")
    print ("      pd.read_table() into 'df':", PATH_TO_TCGA_DATA)
except:
    print ("    ERROR:  File(s) not found")
    #quit()
    
print("  checking dimensions of the file loaded into df")  
print("    df.shape:", df.shape)
print("[TIMESTAMP]", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# keep a backup copy of what was loaded 
df_backup = df.copy()

In [ ]:
## FOR DEBUG ONLY - manually comment out to reset the loaded df object before any modifications
# df = df_backup

In [ ]:
print("  tallying the 'label' column of the loaded dataframe")
print(df['label'].value_counts())

In [ ]:
print("  tallying the 'label' column of the loaded dataframe(possible after subsetting out less samples)")
print(df['label'].value_counts())

In [ ]:
print("  previewing the top of the file")
print(df.head(2))

In [ ]:
print("  previewing the bottom of the file")
print(df.tail(2))

## Save first two columns (sample_id, label) and drop them 

In [ ]:
print ("  saving non-feature columns (sample_id, label) elsewhere, they need to be dropped from df for other functions")

sample_ids = df['sample_id']
labels = df['label']

df = df.drop('sample_id', axis=1)  # axis=0 refers to vertical/rows
df = df.drop('label', axis=1)      # axis=1 refers to horizontal/columns

print("  checking dimensions of the loaded df after dropping")  
print("    df.shape:", df.shape, "  # should be two columns shorter")

## Perform a 80:20 split on the datasets, proportionally
- (**NOTE**) this split, for all preprocessed combinations, is done outside of this notebook for datasets that are normalized or batch corrected
- [not_used_anymore_but_for_python_ref] 
    * (source for split) https://stackoverflow.com/questions/64353575/train-test-split-preserving-class-proportions-in-each-split
    * (source for subsetting df with list of indices) https://stackoverflow.com/questions/19155718/select-pandas-rows-based-on-list-index

In [ ]:
print("  [TCGA dataset already split] assigning the TCGA data into 80% train and 20% test sets")  
percent = 0.80

X_train = df.head(int(len(df)*percent))
Y_train = labels.head(int(len(df)*percent))
id_train= sample_ids.head(int(len(df)*percent))
X_test = df.iloc[max(X_train.index)+1:]
Y_test = labels.iloc[max(Y_train.index)+1:]
id_test= sample_ids.iloc[max(Y_train.index)+1:]
    
print("  checking dimensions of the new variables after train-test split")  
print("    X_train.shape:", X_train.shape, "  # should be 80% of data")
print("    Y_train.shape:", Y_train.shape, "      # should be 80% of labels")
print("    id_train.shape:", id_train.shape, "    # should be 80% of labels")
print("    X_test.shape :", X_test.shape,  "   # should be 20% of data")
print("    Y_test.shape :", Y_test.shape,  "       # should be 20% of labels")    
print("    id_test.shape :", id_test.shape,  "     # should be 20% of labels")    

In [ ]:
print("  Creating the label encoder which is required for ML algorithms to know the ground truth")
label_encoder = LabelEncoder().fit(Y_train)

In [ ]:
print("  previewing 'X_train' that contains the training set data")
print(X_train.head(2))

In [ ]:
print("  tallying the different number of 'labels' among training set data")
print(Y_train.value_counts())

In [ ]:
print("  previewing 'Y_train' that contains the training set labels")
print(Y_train.head(2))

In [ ]:
print("  previewing 'X_test' that contains the test set data")
print(X_test.head(2))

In [ ]:
print("  tallying the different number of 'labels' among test set data")
print(Y_test.value_counts())

In [ ]:
print("  previewing 'Y_test' that contains the test set labels")
print(Y_test.head(2))

## Function declarations

### (Helper functions)

In [ ]:
# (returns) if found then print the (rows=sample_id) and (cols=gene names) that are zero

def check_if_any_genes_zero (arg_data, arg_sample_ids, arg_debug_filename):
    print("  begin check_if_any_genes_zero()")
    
    # remember that arg_data has first two cols as sample_id, label, then rest are genes; rows are each sample
    
    # work with a copy just in case and also assign row names as first column
    tmp_df = arg_data.copy()
    id_column = pd.DataFrame(data = arg_sample_ids)
    tmp_df.index = id_column['sample_id']
    
    print("")
    print("[DEBUG]---------------------------------[DEBUG]")
    
    counter = 0
    
#     ## working - checks all elements and prints out the location with zero value
#     for idx in arg_data.index:
#         for column in arg_data.columns:
#             if arg_data.loc[idx, column] == 0:
#                 print(f'  + found zero @ ({idx},{column}): {arg_data.loc[idx, column]}')

#     # (for testing only) set all value of this gene equal to zero
#     arg_data['ENSG00000000419'] = 0
    
    
    ## working - checks for columns(=genes) where all values zero and prints out the name of gene
    for column in tmp_df.columns:
        if (tmp_df[column] == 0).all():
            # print(f'  + found all zeroes in col @ {column}')
            counter = counter + 1
        
    print(f'  + [Total genes found with all zeros] {counter}')   
    
#     # be sure that you want to save the snapshots of files, because each one is many GBs and takes up lots of space!    
#     print(f'  + writing {arg_debug_filename} to file under results/debug/')
#     words = [PATH_TO_DEBUG, arg_debug_filename, ".tsv"]
#     save_filename_w_path_debug = "".join(words)
#     tmp_df.to_csv(save_filename_w_path_debug, sep="\t")
    
    print("[END_DEBUG]-------------------------[END_DEBUG]")
    print("")
    
    
    print("  end check_if_any_genes_zero()")

###     (Scaling function)

In [ ]:
# (consider; this scaling might be done before other normalization steps? would need to be done outside this notebook)
    
# (returns) scaled version of data passed it; the scaling factors used   
# (source) https://www.geeksforgeeks.org/python-how-and-where-to-apply-feature-scaling/
# Scale the dataset; This is very important before you apply PCA
# Also very important that the same scaling factor is used in both sets, otherwise get erroreous test set
def scale_this_data(arg_data, arg_use_scalar, arg_scale_required): 
    
    print("  begin scale_this_data()")
    print("   checking statistics of data before scaling")  
    print("    mean:", np.mean(arg_data))
    print("    max :", np.amax(arg_data))
    print("    min :", np.amin(arg_data))
#     print("    std :", np.std(arg_data))

    sc = None
    
    if arg_scale_required == True:
        
        print("   SCALE_DATA equals True, so we are going to scale data")  
        
        if arg_use_scalar == None:
            print("   No previous scaling factors, so will create new one and fit it to data")
            
            if SCALER_TYPE == "StandardScaler":
                print("   SCALER_TYPE is 'StandardScaler', now creating an instance of StandardScaler")
                sc = StandardScaler()
            elif SCALER_TYPE == "MinMaxScaler":
                print("   SCALER_TYPE is 'StandardScaler', now creating an instance of MinMaxScaler")
                sc = MinMaxScaler(feature_range=(0,1))
            
            sc.fit(arg_data)   
        else:
            print("   Previous scalar passed into function, so will use it")
            sc = arg_use_scalar

        print("   scaling the data now")
        data_after_scaling = sc.transform(arg_data)
        
        
    else:
        print("   SCALE_DATA equals False, so we are NOT going to scale data")  
        data_after_scaling = arg_data
        
        if arg_use_scalar == None:
            print("   No previous scaling factors, so will create new one but it wont be used")
            
            if SCALER_TYPE == "StandardScaler":
                print("   SCALER_TYPE is 'StandardScaler', now creating an instance of StandardScaler")
                sc = StandardScaler()
            elif SCALER_TYPE == "MinMaxScaler":
                print("   SCALER_TYPE is 'MinMaxScaler', now creating an instance of MinMaxScaler")
                sc = MinMaxScaler(feature_range=(0,1))
            

    print("  checking statistics of data after scaling")  
    print("    mean:", np.mean(data_after_scaling))
    print("    max :", np.amax(data_after_scaling))
    print("    min :", np.amin(data_after_scaling))
    # print("    std :", np.std(data_after_scaling))

    print("  end scale_this_data()")
    
    return pd.DataFrame(data = data_after_scaling), sc

###     (Dimension Reduction functions)

(ex. **reducers**) PCA, UMAP

In [ ]:
# (returns) modified DF and the reduce object (that was used for umap.transform())
# (source on UMAP implementation) https://umap-learn.readthedocs.io/en/latest/basic_usage.html
# (source on PCA implementation) https://www.datacamp.com/tutorial/principal-component-analysis-in-python
# (source on explained variance) https://vitalflux.com/pca-explained-variance-concept-python-example/
# (source on tick marks) https://stackoverflow.com/questions/30914462/how-to-force-integer-tick-labels

def apply_feature_reduction(arg_data, arg_labels, arg_use_reducer, arg_reducer_type, arg_report_suffix):  
    print("  begin apply_feature_reduction() with reducer_type:", arg_reducer_type)
    print("   checking dimensions of data before reducer")       
    print("    arg_data.shape:", arg_data.shape)    
    print("    arg_labels.shape:", arg_labels.shape)                      
    print("   checking statistics of data before reducer")  
    print("    mean:", np.mean(arg_data))
    print("    max :", np.amax(arg_data))
    print("    min :", np.amin(arg_data))
#     print("    std :", np.std(arg_data))

    reducer = None
    exp_var_pca = None

    if arg_use_reducer == None:
        print("   No previous reducer object, so will create new one and fit it to data")
        
        
        if arg_reducer_type == "umap":
            reducer = umap.UMAP(

                random_state=SEED_FOR_RANDOM_STATE
             )
            reducer.fit(arg_data)       # this call updates the `pca` object  
        elif arg_reducer_type == "pca":
            reducer = PCA(
                EXPLAINED_VARIANCE_PERCENT * 0.01,   # here we explicitly mention how much variance we would like PCA 
                                   #to capture and hence, the n_components will vary based on the variance parameter.
                random_state=SEED_FOR_RANDOM_STATE
             )
            reducer.fit(arg_data)       # this call updates the `pca` object   

            print("   applying pca.transform on new pca object")
            data_after_reducer = reducer.transform(arg_data)
            
            print("   now have the number of features to reduce to: ", str(data_after_reducer.shape[1]))
            
            print("   rebuilding the pca object with n_features (important for applying on test data later) ")
            reducer = PCA(
                n_components = data_after_reducer.shape[1],
                random_state=SEED_FOR_RANDOM_STATE
            )
            reducer.fit(arg_data)       # this call updates the `pca` object   
            
            print("   applying pca.transform on new pca object (again, to be sure)")
            data_after_reducer = reducer.transform(arg_data)

            print("   Generate the scree plot ")
            # Determine explained variance using attribute of pca object
            exp_var_pca = reducer.explained_variance_ratio_

            # Cumulative sum of eigenvalues; This will be used to create step plot
            # for visualizing the variance explained by each principal component.
            cum_sum_eigenvalues = np.cumsum(exp_var_pca)

            # Create the visualization plot for explained variaince 
            #   - barplot is used to represent individual explained variances 
            #   - step plot is used to represent the variance explained by different principal components
            fig, ax = plt.subplots(1, figsize=(7, 5))
            plt.bar(
                    range(0,len(exp_var_pca)), 
                    exp_var_pca, 
                    alpha=0.5, 
                    align='center', 
                    label='Individual explained variance'
            )
            plt.step(
                    range(0,len(cum_sum_eigenvalues)), 
                    cum_sum_eigenvalues, 
                    where='mid',
                    label='Cumulative explained variance',
                    color='darkgreen'
                    )
            plt.ylabel('Explained variance ratio')    
            plt.ylim((0,1))
            plt.yticks(np.arange(0, 1, 0.1))
            plt.xlabel('Principal component index')
            ax.xaxis.set_major_locator(MaxNLocator())
            ax.yaxis.set_major_locator(MaxNLocator())
            ax.margins(y=.1, x=.1)
            plt.legend(loc='best')
            plt.tight_layout()
            
            print("   Saving a figure for pca_plot_of_explained_variance")
            pca_plot_filename_wo_extension = "_pca_plot_of_explained_variance"
            
            # save set of figures as pdfs without titles as pdfs for manuscript
            plt.savefig(PATH_TO_PCA_PLOTS_WITHOUT_A_TITLE + pca_plot_filename_wo_extension + ".pdf", bbox_inches='tight')
            
            # save one set of figures as jpgs with titles that are useful in ppt slides
            plt.title("{}% of the variance explained by {} principal components".format(
                                                            str(EXPLAINED_VARIANCE_PERCENT),
                                                            data_after_reducer.shape[1],), 
                           fontsize=14
                     )
            plt.savefig(PATH_TO_PCA_PLOTS + pca_plot_filename_wo_extension + ".jpg", bbox_inches='tight')

            plt.close()
        
        else:
            print("ERROR: Not implemented ")
        
        print("   applying reducer.transform on new reducer object")
        data_after_reducer = reducer.transform(arg_data)
        
       
    else:
        print("   Previous reducer object passed into function, so will use it")
        reducer = arg_use_reducer
        print("   applying reducer.transform on passed in reducer object")
        data_after_reducer = reducer.transform(arg_data)
        
    

    # Create the visualization plot 
    fig, ax = plt.subplots(1, figsize=(7, 5))
    figure_labels = pd.Series(pd.Categorical(arg_labels, categories=reversed(np.unique(labels)), ordered=True))
#     common_cmap = matplotlib.cm.get_cmap('tab20')  # will be deprecated
    common_cmap = matplotlib.colormaps['tab20']
    common_norm = plt.Normalize(vmin=0, vmax=len(np.unique(labels))-1)
    scatter = plt.scatter(
                data_after_reducer[:, 0],                 # x-axis data
                data_after_reducer[:, 1],                 # y-axis data
                s= 20,                                    # marker size
                c=figure_labels.cat.codes.astype(float),     # list of colors
                norm=common_norm,                         # normalization method used to scale scalar data to [0,1] 
                cmap=common_cmap,                         # Colormap instance used to map scalar data to colors
                alpha=0.5
               )

    cbar = fig.colorbar(scatter, 
                    ax=ax, 
                    boundaries=np.arange(figure_labels.cat.categories.size+1)-0.5,
                    orientation='vertical'
                   )
    cbar.set_ticks(np.arange(len(np.unique(labels))))
    cbar.set_ticklabels(figure_labels.cat.categories)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.margins(y=.1, x=.1)

    if arg_reducer_type == "umap":
        print("   Saving a figure for umap:", arg_report_suffix)
        
        plt.xlabel ("UMAP1")
        plt.ylabel ("UMAP2")
        
        # save set of figures as pdfs without titles as pdfs for manuscript
        plt.savefig(PATH_TO_UMAP_PLOTS_WITHOUT_A_TITLE + "_umap_plot_of_"+arg_report_suffix +".pdf")

        # save one set of figures as jpgs with titles that are useful in ppt slides
        plt.title("UMAP projection of " + arg_report_suffix, fontsize=12);
        plt.savefig(PATH_TO_UMAP_PLOTS + "_umap_plot_of_"+arg_report_suffix +".jpg", bbox_inches='tight')
        plt.close()
        
    elif arg_reducer_type == "pca":
        exp_var_pca = reducer.explained_variance_ratio_
        
        print("   Saving a figure for pca_plot_of_PC1_vs_PC2:", arg_report_suffix)
        
        plt.xlabel ("PC1 (" + str(int(exp_var_pca[0]*100)) + "%)")
        plt.ylabel ("PC2 (" + str(int(exp_var_pca[1]*100)) + "%)")
        
        # save set of figures as pdfs without titles as pdfs for manuscript
        plt.savefig(PATH_TO_PCA_PLOTS_WITHOUT_A_TITLE + "_pca_plot_of_"+arg_report_suffix +"_PC1_vs_PC2.pdf")

        # save one set of figures as jpgs with titles that are useful in ppt slides        
        plt.title("PC1 vs PC2 of " + arg_report_suffix, fontsize=12);
        plt.savefig(PATH_TO_PCA_PLOTS + "_pca_plot_of_"+arg_report_suffix +"_PC1_vs_PC2.jpg", bbox_inches='tight')
        plt.close()
        
    else:
        print("ERROR: Not implemented ")
    
    
    
    print("   checking dimensions of data after reducer")       
    print("    data_after_reducerpca.shape: ", data_after_reducer.shape)    
    print("    *notice* # of features reduced to: ", data_after_reducer.shape[1]) 
    print("    arg_labels.shape: ", arg_labels.shape)        
    print("   checking statistics of data after reducer")  
    print("    mean: ", np.mean(data_after_reducer))
    print("    max : ", np.amax(data_after_reducer))
    print("    min : ", np.amin(data_after_reducer))
#     print("    std :", np.std(data_after_reducer))        
    print("  end apply_feature_reduction() with of type ",arg_reducer_type )

    # return the reducer-applied dataset and reducer object
    return pd.DataFrame(data = data_after_reducer), reducer


In [ ]:
# Creates classifier models on training dataset (80% TCGA) 
# - updated one that calls to scaling and feature reduction

# (source for passing in multiple variables via **best_params) 
# https://yasoob.me/2013/08/04/args-and-kwargs-in-python-explained/

# how called
"""
    models = build_classifier_model(X_train_after_DR)
"""

def build_classifier_model(arg_train_data, arg_train_labels, arg_train_ids):    
    encoded_y = pd.DataFrame(data={"Type":label_encoder.transform(arg_train_labels)})  #Y_train here was 'target' originally
    best_gs_models = [None] * grids_n 
    
    # need to keep track of these for later
    current_best_model = []
    current_best_scalar = []
    current_best_umap = []
    current_best_pca = []
#     current_best_model = None     #changing from single best model to best models of each outer folder
#     current_best_scalar = None
#     current_best_umap = None
#     current_best_pca = None

    
    # Construct pipelines for each algorithm
    pipe_svm = Pipeline([('clf', SVC(probability = True, 
                                     random_state=SEED_FOR_RANDOM_STATE, 
                                     break_ties=True, 
                                     decision_function_shape='ovr',
                                     class_weight='balanced',
                                     cache_size=1000)
                         )])
#     pipe_svm = Pipeline([('clf', SVC(probability = True, 
#                                      random_state=SEED_FOR_RANDOM_STATE, 
#                                      break_ties=True, 
#                                      decision_function_shape='ovr', 
#                                      cache_size=1000, 
#                                      tol=1e-4)
#                          )])


    # Initialize the parameters required for the models
    grid_params_svm = [{'clf__kernel': ['rbf', 'linear'],
                        'clf__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
#                         'clf__gamma': [0.0001, 0.001, 0.01, 0.1, 1.0]
                       }]
#     grid_params_svm = [{'clf__kernel': ['linear','rbf'],
#                         'clf__C': [0.5,1,5,7.5,10,20,30,100],
#                         'clf__gamma': [1e-5, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
#                         'clf__class_weight': [None, 'balanced'],
#                        }]

    # create the CV splitter
    outer_k = 5
    inner_k = 5
    inner_cv_skf = StratifiedKFold(n_splits=inner_k, shuffle=True,  random_state=SEED_FOR_RANDOM_STATE)
    outer_cv_skf = StratifiedKFold(n_splits=outer_k, shuffle=True,  random_state=SEED_FOR_RANDOM_STATE)
   

    # outer CV to estimate generalization error     
    non_nested_scores = np.zeros((grids_n, outer_k))
    nested_scores = np.zeros((grids_n, outer_k))
    
    # create new files to keep track of cross validation results on train data
    with open(PATH_TO_CV_RESULTS+"_SVM_build_classifier_results_of_inner_fold.txt",'w') as f:
        f.write("--------------------------")
        f.write("--- File Created at:  " + current_date_time) 
        f.write("--------------------------\n\n")
    with open(PATH_TO_CV_RESULTS+"_SVM_build_classifier_results_of_outer_fold.txt",'w') as f:
        f.write("--------------------------")
        f.write("--- File Created at:  " + current_date_time) 
        f.write("--------------------------\n\n")
        
    outer_k_index = 0
    for outer_train_index, outer_holdout_index in outer_cv_skf.split(X=arg_train_data, y=encoded_y):
        print("[DEBUG] starting outer_fold -- {} of {}". format(outer_k_index+1, outer_k))
        print("  [TIMESTAMP]", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

        cv_X_train = arg_train_data.iloc[outer_train_index]
        cv_Y_train = encoded_y.iloc[outer_train_index]
        cv_Y_train_unencoded =  label_encoder.inverse_transform(cv_Y_train.values.ravel())
        cv_id_train= arg_train_ids.iloc[outer_train_index]
        
        cv_X_test = arg_train_data.iloc[outer_holdout_index]
        cv_Y_test = encoded_y.iloc[outer_holdout_index]
        cv_Y_test_unencoded =  label_encoder.inverse_transform(cv_Y_test.values.ravel())
        cv_id_test= arg_train_ids.iloc[outer_holdout_index]
        
        # apply scaling 
        print (" calling scale_this_data() on cv_X_train")
#         check_if_any_genes_zero(cv_X_train, cv_id_train, "notebook11_tcga_cvtrain_before_scaling")
        cv_X_train, cv_scalar = scale_this_data(cv_X_train, None, SCALE_DATA)
#         check_if_any_genes_zero(cv_X_train, cv_id_train, "notebook11_tcga_cvtrain_after_scaling")
    
        print (" calling scale_this_data() on cv_X_test")
#         check_if_any_genes_zero(cv_X_test, cv_id_test, "notebook11_tcga_cvtest_before_scaling")
        cv_X_test, cv_scalar = scale_this_data(cv_X_test, cv_scalar, SCALE_DATA)
#         check_if_any_genes_zero(cv_X_test, cv_id_test, "notebook11_tcga_cvtest_after_scaling")
        
        # apply dimension reduction
        report_suffix = "trainSet_randomOutOf" + str(outer_k) + "Folds"
        
# -------------- either use this block here       
#         print (" calling apply_feature_reduction() on cv_X_train - UMAP as main reducer")
#         print ("   (for pca)  just to generate plots, discard pca-transformed data")
#         _, cv_pca = apply_feature_reduction(cv_X_train, cv_Y_train_unencoded, None,"pca", report_suffix)
#         print ("   (for umap) will generate plots, keep umap-transformed data")
#         cv_X_train, cv_umap = apply_feature_reduction(cv_X_train, cv_Y_train_unencoded, None,"umap", report_suffix)
#         print (" calling apply_feature_reduction() on cv_X_test")
#         print ("   (for pca)  just to generate plots, discard pca-transformed data")
#         _, cv_pca = apply_feature_reduction(cv_X_test, cv_Y_test_unencoded, cv_pca,"pca", report_suffix)
#         print ("   (for umap) will generate plots, keep umap-transformed data")
#         cv_X_test, cv_umap = apply_feature_reduction(cv_X_test, cv_Y_test_unencoded, cv_umap,"umap", report_suffix)
        
# -------------- Or this block here   
        print (" calling apply_feature_reduction() on cv_X_train - PCA as main reducer")
        print ("   (for umap) just to generate plots, discard umap-transformed data")
        _, cv_umap = apply_feature_reduction(cv_X_train, cv_Y_train_unencoded, None,"umap", report_suffix)
        print ("   (for pca)  will generate plots, keep pca-transformed data")
        cv_X_train, cv_pca = apply_feature_reduction(cv_X_train, cv_Y_train_unencoded, None,"pca", report_suffix)
        print (" calling apply_feature_reduction() on cv_X_test")
        print ("   (for umap) just to generate plots, discard umap-transformed data")
        _, cv_umap = apply_feature_reduction(cv_X_test, cv_Y_test_unencoded, cv_umap,"umap", report_suffix)
        print ("   (for pca)  will generate plots, keep pca-transformed data")
        cv_X_test, cv_pca = apply_feature_reduction(cv_X_test, cv_Y_test_unencoded, cv_pca,"pca", report_suffix)
        
        
        #Construct the grid searches
        gs_svm = GridSearchCV(
                              estimator=pipe_svm, 
                              param_grid=grid_params_svm, 
                              scoring='f1_weighted',
                              cv=inner_cv_skf.split(cv_X_train, cv_Y_train.values.ravel()), 
                              n_jobs=MAX_JOBS, 
                              verbose=1,        # this int can be up to 10; higher value means more output
                              return_train_score=True, 
                              refit=True    # when refit=True; then can use gs.best_estimator_
#                               refit=False
                             )

        # Maintaining the models name and pipeline name to loop over and save the results
        grids = [gs_svm]
 
        for idx, gs in enumerate(grids):
            print(" [DEBUG] Start of loop for ML algorithm type")
            print("   [TIMESTAMP]", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            print('  Classifier: %s' % grids_dict[idx])
            # Fit grid search
            gs.fit(cv_X_train, cv_Y_train.values.ravel())
            print("  Done fitting")
            # Best params
            print('  Best params: %s' % gs.best_params_)
            # Best validation data f1
            print('  Best validation F1: %.3f' % gs.best_score_)

            meantrainingscore = gs.cv_results_['mean_train_score'][gs.best_index_]
            stdtrainingscore = gs.cv_results_['std_train_score'][gs.best_index_]
            meanvalidationscore = gs.cv_results_['mean_test_score'][gs.best_index_]
            stdvalidationscore = gs.cv_results_['std_test_score'][gs.best_index_]
            
            # print(gs.best_estimator_.get_params().keys())
            inner_k_index=0
            
            print("    [DEBUG] outer fold starting series of inner folds to figure out hyperparameters")
            for inner_train_index, validation_index in inner_cv_skf.split(cv_X_train, cv_Y_train):
                print("     [DEBUG] inside outer_fold -- {} of {} --  inner_fold {} of {}". format(outer_k_index+1, outer_k, inner_k_index+1, inner_k))

                gs.estimator.set_params(**gs.best_params_).fit(cv_X_train.iloc[inner_train_index], cv_Y_train.iloc[inner_train_index].values.ravel()) 
                ypred = gs.estimator.predict(cv_X_train.iloc[validation_index])                      #https://stackoverflow.com/questions/51091132/pandas-and-scikit-learn-keyerror-not-in-index
                #print("  [DEBUG]ypred: {}".format(ypred))
                training_report_cv = classification_report(label_encoder.inverse_transform(cv_Y_train.iloc[validation_index].values.ravel()), label_encoder.inverse_transform(ypred))
                f1_cv_macro = f1_score(cv_Y_train.iloc[validation_index].values.ravel(), ypred, average='macro')
                f1_cv_weighted = f1_score(cv_Y_train.iloc[validation_index].values.ravel(), ypred, average='weighted')

                with open(PATH_TO_CV_RESULTS+"_SVM_build_classifier_results_of_inner_fold.txt",'a+') as f:
                    f.write(" +++++ inner fold k=" +str(inner_k_index+1) + " of outer fold k=" +str(outer_k_index+1) + " ++++ \n")  
                    f.write(grids_dict[idx] + " CLASSIFICATION REPORT during CV on TRAIN DATA #{} \n".format(outer_k_index+1))
                    f.write(str(training_report_cv) + "\n")
#                     f.write(grids_dict[idx] + " PERFORMANCE METRIC (f1-macro)    during CV on TRAIN DATA #{}: {} \n".format(
#                                                                                     outer_k_index+1,
#                                                                                     str(f1_cv_macro))
                    f.write(grids_dict[idx] + " PERFORMANCE METRIC (f1-weighted) during CV on TRAIN DATA #{}: {} \n".format(
                                                                                    outer_k_index+1,
                                                                                    str(f1_cv_weighted)))
                            
                       
                            
                    f.write("----------------------------------\n\n")
                    
                inner_k_index += 1
                
                print("     [DEBUG]   end of inner loop iteration -- ")
            print("    [DEBUG] done with this series of inner folds")

                        
            # Predict on test data with best params
            gs.estimator.set_params(**gs.best_params_).fit(cv_X_train, cv_Y_train.values.ravel()) 
            cv_Y_pred = gs.estimator.predict(cv_X_test)
            
            # Performance metrics
            f1_macro = str(f1_score(cv_Y_test, cv_Y_pred, average='macro'))   # no longer used        
            f1_weighted = str(f1_score(cv_Y_test, cv_Y_pred, average='weighted')) 
            acc_balanced  = str(balanced_accuracy_score( cv_Y_test, cv_Y_pred))
            matthews_coef = str(matthews_corrcoef(       cv_Y_test, cv_Y_pred))
      
            print("    Evaluation set for best params -- F1 score (weighted): {} ".format(f1_weighted))
            print("    Evaluation set for best params -- Balanced ACC: {} ".format(acc_balanced))
            print("    Evaluation set for best params -- Matthews Coef: {} ".format(matthews_coef))

            # Print and Save test result on evaluation data
            print("    Saving results to file at path:" + PATH_TO_CV_RESULTS)
            test_report_evaluation = classification_report(label_encoder.inverse_transform(cv_Y_test.values.ravel()), label_encoder.inverse_transform(cv_Y_pred))              #https://joshlawman.com/metrics-classification-report-breakdown-precision-recall-f1/
            test_con_matrix_evaluation = confusion_matrix(label_encoder.inverse_transform(cv_Y_test.values.ravel()), label_encoder.inverse_transform(cv_Y_pred))                    #https://stats.stackexchange.com/questions/95209/how-can-i-interpret-sklearn-confusion-matrix
            with open(PATH_TO_CV_RESULTS+"_SVM_build_classifier_results_of_outer_fold.txt",'a+') as f:
                f.write(" +++++ best fold of ++ of outer fold k=" +str(outer_k_index+1) + "++++ \n")  
                f.write(grids_dict[idx]+" CLASSIFICATION REPORT during CV on TRAIN DATA #{}\n".format(
                                                                                    outer_k_index+1))
                f.write(str(test_report_evaluation) + "\n" )
#                 f.write(grids_dict[idx]+" PERFORMANCE METRIC (f1-macro)    during CV on TRAIN DATA: "  + str(f1_macro) + "\n")
#                 f.write("grids_dict[idx]+" "PERFORMANCE METRIC (f1-weighted) during CV on TRAIN DATA #{}: {} \n\n".format(
#                                                                                     outer_k_index+1,
#                                                                                     str(f1_weighted)))
    
                f.write("OVERALL PERFORMANCE METRICS ON TRAIN DATA -- outer fold CV #{} \n".format(outer_k_index+1))
                f.write("  f1-weighted:   "  + f1_weighted + "\n")
                f.write("  acc-balanced:  "  + acc_balanced + "\n")
                f.write("  matthews_coef: "  + matthews_coef + "\n")
                f.write("\n")     
    
    
                f.write(grids_dict[idx]+" CONFUSION MATRIX" + "\n")
                f.write(str(test_con_matrix_evaluation) + '\n\n')
                f.write("----------------------------------\n\n")
            nested_scores[idx][outer_k_index] = f1_weighted #should be macro or weighted?
            non_nested_scores[idx][outer_k_index] = meanvalidationscore

            
            # (updated to save every outer_fold )
            # - involves changing the 'current_best_*' variables to be list of size `outer_k_index`

            print("    Saving best model/scalar/umap/pca for outer_fold #{}". format(outer_k_index+1))
#                 best_gs_models[idx] = gs.estimator
#                 current_best_model = gs.estimator
            current_best_model.append(gs.best_estimator_)  
            current_best_scalar.append( cv_scalar)
            current_best_umap.append(cv_umap)
            current_best_pca.append(cv_pca)
            print("      current_best_model :", current_best_model[outer_k_index])     
            print("        n_features:", current_best_model[outer_k_index].n_features_in_)        
            print("      current_best_scalar :", current_best_scalar[outer_k_index])      
            print("      current_best_umap :", current_best_umap[outer_k_index])     
            print("        n_features:", cv_X_test.shape[1]) 
            
            
#             if (f1_weighted == max(nested_scores[idx,:])): 
#                 print("    Saving this model, scalar, and reducer objects since it has current max score of nested cv")
# #                 best_gs_models[idx] = gs.estimator
# #                 current_best_model = gs.estimator
#                 current_best_model = gs.best_estimator_  # this only works when refit parameter is true
#                 current_best_scalar = cv_scalar
#                 current_best_umap = cv_umap
#                 current_best_pca = cv_pca
#                 print("      current_best_model :", current_best_model)     
#                 print("        n_features:", current_best_model.n_features_in_)        
#                 print("      current_best_scalar :", current_best_scalar)      
#                 print("      current_best_umap :", current_best_umap)     
#                 print("        n_features:", cv_X_test.shape[1]) 
             
            print(" [DEBUG] End of loop for ML algorithm type -- this currently just SVM ")
            
        outer_k_index += 1
        print("[DEBUG] End of outer loop iteration -- \n\n\n\n")
    
    return current_best_model, current_best_scalar, current_best_umap, current_best_pca

In [ ]:
# how called
""" 
    # this is a void function, expect new files created in folder: classification_PREDICTIONS
    apply_model( _TODO___)
"""
# (some sources) 
# - #https://joshlawman.com/metrics-classification-report-breakdown-precision-recall-f1/
# - #https://stats.stackexchange.com/questions/95209/how-can-i-interpret-sklearn-confusion-matrix

#APPLY MODEL TO DATASETS TO ASSESS TESTING CLASSIFICATION ACCURACY
def apply_model_to_assess_performance(arg_outer_k_index, arg_best_model, arg_best_scalar, arg_best_umap, arg_best_pca, arg_test_data, arg_test_labels, arg_test_ids, arg_report_suffix):

    print("  checking info of some arguments ")  
    print("    arg_outer_k_index : ", arg_outer_k_index)     
    print("    arg_best_model : ", arg_best_model)     
    print("      n_features: ", arg_best_model.n_features_in_)        
    print("    arg_best_scalar : ", arg_best_scalar)      
    print("    arg_best_umap : ", arg_best_umap)     
    print("    arg_best_pca : ", arg_best_pca)   
    
    # create the label_encoder based on the arg_report_stuff
    #### _TODO_
    # for now assume this
#    label_encoder = LabelEncoder().fit(arg_test_labels)
    
    # create new file to keep track of this result
    write_filename = PATH_TO_PREDICTIONS \
                    + "_SVM_apply_classifier_results_for_" \
                    + arg_report_suffix \
                    + "_" \
                    + str(arg_outer_k_index+1) \
                    + "_" \
                    + ".txt"
            
    with open(write_filename,'w') as f:
        f.write("--------------------------")
        f.write("--- File Created at:  " + current_date_time) 
        f.write("--------------------------\n\n")
        
    # create variables for the excel files too   
    write_excel_name_con_matrix = PATH_TO_PREDICTIONS_EXCEL_FILES \
                    + "con_matrix_for_" \
                    + arg_report_suffix \
                    + "_" \
                    + str(arg_outer_k_index+1) \
                    + "_" \
                    + ".xlsx"
    
    write_excel_name_class_wise_metrics = PATH_TO_PREDICTIONS_EXCEL_FILES \
                    + "classWiseMetrics_for_" \
                    + arg_report_suffix \
                    + "_" \
                    + str(arg_outer_k_index+1) \
                    + "_" \
                    + ".xlsx"
        

    # (consider) try-catch statement that includes reverse of this function to open the .pkl file
    ## joblib.dump(best_gs_models[idx], PATH_TO_MODEL_NAME + '_' + grids_dict[idx] + '.pkl', compress=1)

    # apply scaling and dimension reduction here with best scalar and pca objects from above
        
    print (" calling scale_this_data() from apply_model_to_assess_performance() on", arg_report_suffix) 
    check_if_any_genes_zero(arg_test_data, arg_test_ids, "".join(["notebook11_",arg_report_suffix,"_","before_scaling"]))
    test_data_after_scalar, best_scalar = scale_this_data(arg_test_data, arg_best_scalar, SCALE_DATA)
    check_if_any_genes_zero(test_data_after_scalar, arg_test_ids, "".join(["notebook11_",arg_report_suffix,"_","after_scaling"]))

    
    report_suffix = arg_report_suffix
    
    # -------------- either use this block here            
#     print (" calling apply_feature_reduction() from apply_model_to_assess_performance(UMAP) on", arg_report_suffix)
#     print ("   (for pca)  just to generate plots, discard pca-transformed data")
#     _, best_pca = apply_feature_reduction(test_data_after_scalar, arg_test_labels, arg_best_pca,"pca", report_suffix)
#     print ("   (for umap) will generate plots, keep umap-transformed data")
#     test_data_after_reducer, best_umap = apply_feature_reduction(test_data_after_scalar, arg_test_labels, arg_best_umap,"umap", report_suffix)

    # -------------- Or this block here  
    print (" calling apply_feature_reduction() from apply_model_to_assess_performance(PCA) on", arg_report_suffix)
    print ("   (for umap) will generate plots, keep umap-transformed data")
    _, best_umap = apply_feature_reduction(test_data_after_scalar, arg_test_labels, arg_best_umap,"umap", report_suffix)
    print ("   (for pca)  just to generate plots, discard pca-transformed data")
    test_data_after_reducer, best_pca = apply_feature_reduction(test_data_after_scalar, arg_test_labels, arg_best_pca, "pca", report_suffix)

    
    
    print("    Making the prediction with the fitted model")
    print("      begin call arg_best_model.predict()")
    test_pred = arg_best_model.predict(test_data_after_reducer)
 
    print("    Preparing the predictions just returned and ground truth label values to be in same format")
    predicted_labels = pd.DataFrame(data = label_encoder.inverse_transform(test_pred))
    ground_truth = pd.DataFrame(data = arg_test_labels)

    print("    Generating results using sklearn modules for predictions compared to ground truth labels")
    test_report   = classification_report(       ground_truth, predicted_labels)  
    f1_macro      = str(f1_score(                ground_truth, predicted_labels, average='macro'))
    f1_weighted   = str(f1_score(                ground_truth, predicted_labels, average='weighted'))
    acc_balanced  = str(balanced_accuracy_score( ground_truth, predicted_labels))
    matthews_coef = str(matthews_corrcoef(       ground_truth, predicted_labels))
    con_matrix  = confusion_matrix(ground_truth, predicted_labels, labels=np.unique(labels))      
    con_matrix = np.float64(con_matrix)
    
#     print("    Creating merged_array of predictions and truth values to know size of confusion matrix")
#     merged_array_labels_of_pred_and_truth = []
#     merged_array_labels_of_pred_and_truth.extend(np.unique(label_encoder.inverse_transform(test_pred)))
#     merged_array_labels_of_pred_and_truth.extend(np.unique(arg_test_labels.to_numpy()))
#     print("      (# unique labels in test_pred) ", np.unique(label_encoder.inverse_transform(test_pred)))
#     print("      (# unique labels in arg_test_labels) ", np.unique(arg_test_labels.to_numpy()))
#     print("      (# unique labels in merged_array) ", np.unique(merged_array_labels_of_pred_and_truth))
    
    print("    Saving single confusion matrix as excel table for primary/derived metrics")
    # convert the sklearn confusion matrix object into a dataframe that can be written to an excel file
    # (source) https://stackoverflow.com/questions/60841348/how-to-convert-confusion-matrix-to-dataframe
    df_con_matrix = pd.DataFrame(con_matrix, columns = np.unique(labels))    
    df_con_matrix.to_excel(write_excel_name_con_matrix)
    
    print("    Exploring the confusion matrix for primary/derived metrics")
    # (source) https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class
    results_metrics = []
    results_derived = []
    results_derived2 = []
    results_both_metrics = []
    
    for class_id in label_encoder.transform(np.unique(labels)):
        
        class_label = np.unique(labels)[class_id]   
        print ("      (working on class_label) ", class_label)
        
        # figure out how many of this class there is
        # (source) https://stackoverflow.com/questions/49471442/using-pandas-value-counts-to-get-one-value
        test_labels_df =  pd.DataFrame(arg_test_labels, columns = ['label'])  
        countDat = test_labels_df['label'].value_counts()
        try:
            class_count =  countDat[class_label]      
        except:
            class_count = 0   # in situations where the label isn't predicted or actual, need this try-except block                               
        print("         print(class_count):", class_count)

        
        # figure out primary metrics
        TP = con_matrix[class_id,class_id]
        FN = np.sum(con_matrix[class_id]) - TP
        FP = np.sum(con_matrix[:,class_id]) - TP
        TN = np.sum(con_matrix) - TP - FN - FP

        # build matrix for primary metrics counts 
        results_metrics.append([
                                        class_label,
                                        class_count,
                                        TP,
                                        FN,
                                        FP,
                                        TN
                               ])
        
        
        # figure out derived metrics
        #   sensitivity = 0 if TP == 0
        if TP != 0:
            sensitivity = TP/(TP+FN)
        else:
            sensitivity = 0.
        specificity = TN/(TN+FP)
        FPR = 1 - sensitivity
        FNR = 1 - specificity
        PPV = TP/(TP+FP)
        NPV = TN/(TN+FN)
        accuracy = (TP+TN)/(TP+FP+FN+TN)
        prevalence = (TP+FN)/(TN+TP+FP+FN)
        f_score = 2*(PPV*sensitivity)/(PPV+sensitivity)
        
        # build matrix for derived metrics
        results_derived.append([
                                        class_label,
                                        PPV,
                                        NPV,
                                        accuracy,
                                        prevalence
                               ])
        results_derived2.append([
                                        class_label,
                                        sensitivity,
                                        specificity,
                                        FPR,
                                        FNR,
                                        f_score
                               ])
        
        # build matrix for derived metrics
        results_both_metrics.append([
                                        class_label,
                                        class_count,
                                        TP,
                                        FN,
                                        FP,
                                        TN,
                                        PPV,
                                        NPV,
                                        accuracy,
                                        prevalence,
                                        sensitivity,
                                        specificity,
                                        FPR,
                                        FNR,
                                        f_score
                               ])
        
        print ("        end of iteration on: ", class_label)
    
    print("    Saving single excel table for primary/derived metrics")
    output_both_metrics = pd.DataFrame(results_both_metrics,columns = [
                                        'class',
                                        'count',
                                        'TP',
                                        'FN',
                                        'FP',
                                        'TN',
                                        'PPV',
                                        'NPV',
                                        'accuracy',
                                        'prevalence',
                                        'sensitivity',
                                        'specificity',
                                        'FPR',
                                        'FNR',
                                        'f1-score'
                                                ])  
    output_both_metrics.to_excel(write_excel_name_class_wise_metrics)
    
    print("    Printing the overall performance metrics for "+arg_report_suffix+" #"+str(arg_outer_k_index+1)+" :")
#     print('      F1 score (macro):     ' + f1_macro)
    print('      F1 score (weighted):  ' + f1_weighted)
    print('      ACC-balanced:         ' + acc_balanced)
    print('      MCC (matthews_coef):  ' + matthews_coef)
    
    print("    Saving results to file at path:" + PATH_TO_PREDICTIONS)
    
    # create the dataframe objects for each matrix made in the loop, add a header too 
    output_metrics = pd.DataFrame(results_metrics,columns = [
                                        'class',
                                        'count',
                                        'TP',
                                        'FN',
                                        'FP',
                                        'TN'
                                                ])    
    output_derived = pd.DataFrame(results_derived,columns = [
                                        'class',
                                        'PPV',
                                        'NPV',
                                        'accuracy',
                                        'prevalence'
                                                ])    
    output_derived2 = pd.DataFrame(results_derived2,columns = [
                                        'class',
                                        'sensitivity',
                                        'specificity',
                                        'FPR',
                                        'FNR',
                                        'f1-score'
                                                ])    
    
    with open(write_filename,'a+') as f:
        f.write("SVM CLASSIFICATION REPORT on TEST DATA " + arg_report_suffix + " #" + str(arg_outer_k_index+1) + "\n")
        f.write(str(test_report) + "\n" )
        f.write("OVERALL PERFORMANCE METRICS on TEST DATA " + arg_report_suffix + " #" + str(arg_outer_k_index+1) + "\n")
#         f.write("  f1-macro:      "  + f1_macro + "\n")
        f.write("  f1-weighted:   "  + f1_weighted + "\n")
        f.write("  acc-balanced:  "  + acc_balanced + "\n")
        f.write("  matthews_coef: "  + matthews_coef + "\n")
        f.write("\n")
        f.write("CONFUSION MATRIX" + "\n")
        f.write(str(con_matrix) + '\n')
        f.write("\n")
        f.write("INDIVIDUAL PERFORMANCE METRICS (custom calculations) \n")
        f.write(str(output_metrics))
        f.write("\n")
        f.write(str(output_derived))
        f.write("\n")
        f.write(str(output_derived2))
        f.write("\n")
        f.write("----------------------------------\n\n")      
          
    
    # (later) can refer to old code to print out performance metrics; in detail 
    # (later) can refer to old code to print out top 3 best predictions per sample, to investigate
          
    return 
    
        
##### <end of function apply_model()

# Main Function cont.

In [ ]:
grids_n = 1  # svm
grids_dict = {0: 'SVM'}

print (" Calling -- build_classifier_model() to build model/estimator \n")
print("[TIMESTAMP]", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

best_model, best_scalar, best_umap, best_pca = build_classifier_model(X_train, Y_train, id_train) #scaling/DR done at each CV


print("[TIMESTAMP]", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print (" done calling -- build_classifier_model()")
print("\n")

### Main Function --- apply fitted model on test set #1: internal evaluation set of 20% TCGA

In [ ]:
print (" Begin main function block --- apply fitted model on test set #1: internal evaluation set of 20% TCGA")
print("[TIMESTAMP]", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

outer_k = 5

for outer_k_index in range(outer_k):
    print ("  Calling -- apply_model() -- for Test Set #1 on iteration #{} of {}\n".format(outer_k_index+1, outer_k))
    apply_model_to_assess_performance(
                                      outer_k_index,
                                      best_model[outer_k_index], 
                                      best_scalar[outer_k_index], 
                                      best_umap[outer_k_index], 
                                      best_pca[outer_k_index], 
                                      X_test, 
                                      Y_test, 
                                      id_test, 
                                      "testSet01_internalEvaluation"
                                     ) 
    print ("  done calling -- apply_model()")
    print("\n")


print("[TIMESTAMP]", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print (" End main function block --- apply fitted model on test set #1: internal evaluation set of 20% TCGA")



# print ("  Calling -- apply_model() -- for Test Set #1 \n")
# apply_model_to_assess_performance(best_model, best_scalar, best_umap, best_pca, X_test, Y_test, id_test, "testSet01_internalEvaluation") # scaling and DR part of fxn
# print ("  done calling -- apply_model()")
# print("\n")

### Main Function --- prepare test set #2: external independent set of 100% GTEx

In [ ]:
print("  Before next apply_model() call, need to load independent data and parse into variables")

print("  Trying to load gtex data into a Pandas dataframe")  

try:
    df = pd.read_table(PATH_TO_GTEX_DATA)
    
    print ("    File(s) found")
    print ("      pd.read_table() into 'df':", PATH_TO_GTEX_DATA)
except:
    print ("    ERROR:  File(s) not found")
    #quit()
    
print("  checking dimensions of the file loaded into df")  
print("    df.shape:", df.shape)

In [ ]:
print("  tallying the 'label' column of the loaded dataframe")
print(df['label'].value_counts())

In [ ]:
print("  previewing the top of the file")
print(df.head(2))

In [ ]:
print ("  saving non-feature columns (sample_id, label) elsewhere, they need to be dropped from df for other functions")
Y_independent = df['label']
id_independent = df['sample_id']

df = df.drop('sample_id', axis=1)  # axis=0 refers to vertical/rows
df = df.drop('label', axis=1)      # axis=1 refers to horizontal/columns

X_independent = df  # notice we pass it here to match up how rest of code works, just different test set

print("  checking dimensions of the new variables ")  
print("    X_independent.shape :", X_independent.shape,  "   # should be 100% of gtex data")
print("    Y_independent.shape :", Y_independent.shape,  "   # should be 100% of gtex labels")

### Main Function --- apply fitted model on test set #2: external independent set of 100% GTEx¶

In [ ]:
print (" Begin main function block --- apply fitted model on test set #2: external independent set of 100% GTEx")
print("[TIMESTAMP]", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

outer_k = 5

for outer_k_index in range(outer_k):
    print ("  Calling -- apply_model() -- for Test Set #2 on iteration #{} of {}\n".format(outer_k_index+1, outer_k))
    apply_model_to_assess_performance(
                                      outer_k_index,
                                      best_model[outer_k_index], 
                                      best_scalar[outer_k_index], 
                                      best_umap[outer_k_index], 
                                      best_pca[outer_k_index], 
                                      X_independent, 
                                      Y_independent, 
                                      id_independent, 
                                      "testSet02_externalIndependent"
                                     ) # scaling and DR part of fxn
    print ("  done calling -- apply_model()")
    print("\n")


print("[TIMESTAMP]", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print (" End main function block --- apply fitted model on test set #2: external independent set of 100% GTEx")





# print ("  Calling -- apply_model() -- for Test Set #2 \n")
# apply_model_to_assess_performance(best_model, best_scalar, best_umap, best_pca, X_independent, Y_independent, id_independent, "testSet02_externalIndependent")
# print ("  done calling -- apply_model() ")
# print("\n")